# Multichain

A Job can generate new credits with time, by bonding Keep3r Liquidity Pool tokens kLP to it. Liquidities will be handled by the Keep3r Liquidity Pools (`keeep3r.pool`).
Once kLPs are added to a job with addLiquidityToJob, the job starts immediately to mint new KP3R credits, that can be collectable only by the keepers, in reward for working the job. The credit minting system requires no further action from the job owner (`provider`).

### Setup
Run first `00-setup.ipynb` and select its kernel

In [4]:
// create job and add liquidity

next(async()=>{
    job = await(await ethers.getContractFactory('JobForTest')).connect(provider).deploy(keep3r.v2.address)

    await keep3r.v2.connect(provider).addJob(job.address)
    await keep3r.pool.connect(provider).approve(keep3r.v2.address, klpBalance)
    await keep3r.v2.connect(provider).addLiquidityToJob(job.address, keep3r.pool.address, klpBalance)
})

In [5]:
// charts configuration

next(async() =>{
    await $.resetRecording()
    $.resetTraces()

    $.setPeriodTrace(432000)
    
    $.addViewTrace(keep3r.v2, 'totalJobCredits', [job.address])
    $.addViewTrace(keep3r.v2, 'jobPeriodCredits', [job.address]) 
//     TODO: add multiple arguments to view
    $.addEventTrace(keep3r.v2.web3, 'LiquidityCreditsReward', '_rewardedAt')
    $.addEventTrace(keep3r.v2.web3, 'LiquidityAddition')
    $.addEventTrace(keep3r.v2.web3, 'KeeperWork')
    $.addEventTrace(uniV3Pool.web3, 'Swap')    
})

### Reward Periods

To handle KP3R credits minting and quoting, Keep3r introduces reward periods, in which KP3R quote remains stable for each pair, and gas-efficiently processed. These quotes are used within the protocol to mint credits and reward keepers.

The underlying KP3R of the liquidity provided, should generate the same amount of KP3R every inflationPeriod, thereby minting the proportional amount each rewardPeriod as KP3R credits for the job.

In [6]:
// liquidity added in 00-setup

next(async()=>{
    rewardPeriodTime = (await keep3r.v2.rewardPeriodTime()).toNumber()
    inflationPeriod = (await keep3r.v2.inflationPeriod()).toNumber()
    
    console.log('KP3R spent', bnToNumber(kp3rSpent))
    console.log('WETH spent', bnToNumber(wethSpent))
    console.log('kLP minted', bnToNumber(klpBalance))
    console.log('reward period',  rewardPeriodTime / (3600 * 24), 'days')
    console.log('inflation period', (await keep3r.v2.inflationPeriod()).toNumber() / (3600 * 24), 'days')
    console.log('KP3R minted by reward period', bnToNumber(await keep3r.v2.jobPeriodCredits(job.address)))
    console.log('KP3R minted by inflation period', bnToNumber((await keep3r.v2.jobPeriodCredits(job.address)).mul(inflationPeriod).div(rewardPeriodTime)))
})

KP3R spent 9.99
WETH spent 1.3
kLP minted 3.6
reward period 5 days
inflation period 34 days
KP3R minted by reward period 1.48
KP3R minted by inflation period 10.09


To determine the value of a certain liquidity, Keep3r uses a TWAP calculation to get the average quote of a pair in the last completed epoch. The same calculation is applied to quote rewards for keepers (that spend gas in ETH and receive KP3R rewards), using a predefined KP3R/WETH pool as an oracle.

- A job will mint the result of quoteLiquidity every rewardPeriodTime 
- Keep3r will store and use the average quote for the last epochfor each given liquidity
- Remaining credits will be updated to current quotes each time a `rewardPeriod` starts
- Credits older than previous `rewardPeriod` will no longer be updateable and will expire

In [7]:
// credit mining without working
// to make the downfall straighter we have to lessen the hours in time hours
// the higher rewardPeriodTime is, the longer the saw pattern will be

next(async()=>{
    
    console.log('Recording with quote working')
    await $.sleepAndRecord(
        2.5 * rewardPeriodTime,
        $.time(1,'hours')
    )
    
    console.log('Setting the quote to fail')
    await keep3r.helper.setShouldFail(true);
    
    console.log('Recording with quote not working')
    await $.sleepAndRecord(
        2.2 * rewardPeriodTime,
        $.time(1,'hours')
    )
    
    console.log('Setting the quote to work')
    await keep3r.helper.setShouldFail(false);
    
    console.log('Recording with quote working')
    await $.sleepAndRecord(
        2 * rewardPeriodTime,
        $.time(1,'hours')
    )
    
    await $.draw()
})

Recording with quote working
Setting the quote to fail
Recording with quote not working
Setting the quote to work
Recording with quote working
